# CMPS 320
## Lab 7: Logistic Regression

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.linear_model as skl_lm
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing


import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline
plt.style.use('seaborn-white')

### Data Description

A simulated data set containing information on ten thousand customers. The aim here is to predict which customers will default on their credit card debt.

#### Format

A data frame with 10000 observations on the following 4 variables.

***default***

A factor with levels No and Yes indicating whether the customer defaulted on their debt

***student***

A factor with levels No and Yes indicating whether the customer is a student

***balance***

The average balance that the customer has remaining on their credit card after making their monthly payment

***income***

Income of customer

In [2]:
# Load data

Default = pd.read_excel('Default.xlsx', index_col=0)

C:\Users\joshi\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# Obtain summary of the dataframe
Default.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 390.6+ KB


In [4]:
# View the first five rows of the dataframe
Default.head()

,default,student,balance,income
1,No,No,729.526495,44361.625074
2,No,Yes,817.180407,12106.134700
3,No,No,1073.549164,31767.138947
4,No,No,529.250605,35704.493935
5,No,No,785.655883,38463.495879


In [5]:
# Generate descriptive statistics
Default.describe(include='all')

,default,student,balance,income
count,10000,10000,10000.000000,10000.000000
unique,2,2,NaN,NaN
top,No,No,NaN,NaN
freq,9667,7056,NaN,NaN
mean,NaN,NaN,835.374886,33516.981876
std,NaN,NaN,483.714985,13336.639563
min,NaN,NaN,0.000000,771.967729
25%,NaN,NaN,481.731105,21340.462903
50%,NaN,NaN,823.636973,34552.644802
75%,NaN,NaN,1166.308386,43807.729272


In [6]:
# Check the target variable distribution
Default.default.value_counts()

No     9667
Yes     333
Name: default, dtype: int64

There are a total of 10000 elements in the default column, and there are 2 unique values 'No' and 'Yes'.

The number of people who defaulted to "Yes" was 333 -- Only 3.3% of all 10,000 people.

There are more cases of No than cases of Yes. 

When classifying like this, when the number of samples of one label/class is overwhelmingly large/small and thus out of balance with the number of samples of another class, this situation is called class imbalance.

In [7]:
Default.student.value_counts()              

No     7056
Yes    2944
Name: student, dtype: int64

About 30% of students default.

In [8]:
pd.crosstab(Default.student, Default.default)

default,No,Yes
student,,
No,6850,206
Yes,2817,127


### Logistic Regression

Since scikit learn models only allow `numeric` features, category variables must be encoded using dummy variables.

In [9]:
pd.get_dummies(Default).head()

,balance,income,default_No,default_Yes,student_No,student_Yes
1,729.526495,44361.625074,1,0,1,0
2,817.180407,12106.134700,1,0,0,1
3,1073.549164,31767.138947,1,0,1,0
4,529.250605,35704.493935,1,0,1,0
5,785.655883,38463.495879,1,0,1,0


In [10]:
Default_enc = pd.get_dummies(Default, drop_first=True)
Default_enc.head()

,balance,income,default_Yes,student_Yes
1,729.526495,44361.625074,0,0
2,817.180407,12106.134700,0,1
3,1073.549164,31767.138947,0,0
4,529.250605,35704.493935,0,0
5,785.655883,38463.495879,0,0


**Note**: If 'default', default_Yes is 1 and  If 'student', student_Yes is 1

### Logistic Regression Using scikit-learn

In [11]:
# import scikit-learn LogisticRegression estimator
from sklearn.linear_model import LogisticRegression

#### Category variable 'balance' as predictor

In [12]:
# Instantiate the estimator with the solver 'newton-cg' 
logistic_reg = LogisticRegression(solver='newton-cg')

X = Default_enc.balance.values.reshape(-1, 1)  # Since LogisticRegression interfaces with X in 2D, reshape it into an nx1 matrix.

# Default_Yes as Response
y = Default_enc.default_Yes

In [13]:
# Fit the model 
logistic_reg.fit(X, y)

LogisticRegression(solver='newton-cg')

In [14]:
print('classes: ',logistic_reg.classes_)
print('intercept :', logistic_reg.intercept_)
print('coefficient: ',logistic_reg.coef_)       

classes:  [0 1]
intercept : [-10.65132824]
coefficient:  [[0.00549892]]


For the Default data, estimated coefficients of the logistic regression model that predicts the probability of default using balance. 

**Interpretation**: A one-unit increase in balance is associated with an increase in the log odds of default by 0.0055 units.

#### Making Predictiions

In [15]:
X_new = np.array([1000, 2000, 1700]).reshape(-1,1)
logistic_reg.predict_proba(X_new) # request a response from the logistic regression estimator with probability

array([[0.99424785, 0.00575215],
       [0.41423075, 0.58576925],
       [0.78636832, 0.21363168]])

We predict that the default probability for an individual with a balance of $1,000 is 0.00575.

We predict that the default probability for an individual with a balance of $2,000 is 0.5857 which is much higher.

In [16]:
# Request the estimated response as a class. 
y_pred = logistic_reg.predict(X_new) # default threshold is 0.5
y_pred

array([0, 1, 0], dtype=uint8)

In [17]:
threshold = 0.5 # Setting your own threshold
y_pred = (logistic_reg.predict_proba(X_new)[:,1] <= threshold).astype(bool) # set threshold as 0.5
y_pred

array([ True, False,  True])

#### Category variable 'student_Yes' as predictor


In [18]:
logistic_reg = LogisticRegression(solver='newton-cg')  
X = Default_enc.student_Yes.values.reshape(-1, 1)
y = Default_enc.default_Yes
logistic_reg.fit(X, y)
print('classes: ',logistic_reg.classes_)
print('intercept :', logistic_reg.intercept_)
print('coefficient: ',logistic_reg.coef_)

classes:  [0 1]
intercept : [-3.50213151]
coefficient:  [[0.39959759]]


The coefficient corresponding to the student is 0.39, which is positive. That is, students are more likely to default.

***Using the previous model to predict the default when you are a student and when you are not ***

In [19]:
X_new = np.array([1, 0]).reshape(-1,1)
logistic_reg.predict_proba(X_new)

array([[0.95699715, 0.04300285],
       [0.97074836, 0.02925164]])

In [20]:
y_pred = logistic_reg.predict(X_new) # default threshold is 0.5
y_pred

array([0, 0], dtype=uint8)

In both cases, the model predicts 'default' = No (0) if only information on whether it is 'student' or 'not student' is provided as a predictor. 

These predictions are expected, because regardless of whether you are a student or not, most are not 'default'.

The default probability for students is 0.043, which is slightly higher than 0.029 for non-students.

### Multiple Logistic Regression

In [21]:
Default_enc.head(3)

,balance,income,default_Yes,student_Yes
1,729.526495,44361.625074,0,0
2,817.180407,12106.134700,0,1
3,1073.549164,31767.138947,0,0


In [22]:
# Category variables 'balance', 'income', and 'student_Yes' as predictor
X = Default_enc.loc[:, ['balance', 'income', 'student_Yes']]
X['income'] = X['income']*0.001  # income was measured in thousands of dollars

# Default_Yes as Response
y = Default_enc.default_Yes

In [23]:
X.head()

,balance,income,student_Yes
1,729.526495,44.361625,0
2,817.180407,12.106135,1
3,1073.549164,31.767139,0
4,529.250605,35.704494,0
5,785.655883,38.463496,0


In [24]:
# Fit the model 
logistic_reg.fit(X, y)
print('classes: ',logistic_reg.classes_)
print('intercept :', logistic_reg.intercept_)
print('coefficient: ')
list(zip(X.columns, logistic_reg.coef_[0]) )

classes:  [0 1]
intercept : [-10.90180101]
coefficient: 


[('balance', 0.005730606006120194),
 ('income', 0.0039616387914367555),
 ('student_Yes', -0.6125702572679528)]

The negative coefficient for 'student_Yes' in the multiple logistic regression indicates that for a fixed value of balance and income, a student is less likely to default than a non-student.

In [25]:
X_new = np.array([[1500, 40, 1],    # balance 1500, income 40, student
                  [1500, 40, 0]])   # balance 1500, income 40, non-student
logistic_reg.predict_proba(X_new)

array([[0.94047545, 0.05952455],
       [0.89542804, 0.10457196]])

For non-students with a credit card balance of $\$1,500$ and an income of $40,000, the probability of 'default' increased from 0.0595 to 0.105.

## Alternative Method: Multiple Logistic Regression Using statsmodels

In [26]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [27]:
logreg_stats = smf.glm(formula = 'default ~ student + balance + income', 
                       data=Default, family=sm.families.Binomial()).fit()
logreg_stats.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                        Generalized Linear Model Regression Results                        
===========================================================================================
Dep. Variable:     ['default[No]', 'default[Yes]']   No. Observations:                10000
Model:                                         GLM   Df Residuals:                     9996
Model Family:                             Binomial   Df Model:                            3
Link Function:                               logit   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:                -785.77
Date:                             Mon, 15 Nov 2021   Deviance:                       1571.5
Time:                                     12:06:53   Pearson chi2:                 7.00e+03
No. Iterations:                                  9                                         
Covariance Type:                         nonrobust                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         10.8690      0.492     22.079      0.000       9.904      11.834
student[T.Yes]     0.6468      0.236      2.738      0.006       0.184       1.110
balance           -0.0057      0.000    -24.737      0.000      -0.006      -0.005
income         -3.033e-06    8.2e-06     -0.370      0.712   -1.91e-05     1.3e-05
==================================================================================
"""

In [28]:
print(Default.default.value_counts())

No     9667
Yes     333
Name: default, dtype: int64


In [29]:
# If we do not give the fitted model a new predictor, it uses the probability of the response to the training set.
logreg_stats_pred_prob = logreg_stats.predict()
logreg_stats_pred_prob[:10] # Probability of 'Default'

array([0.99857128, 0.9988778 , 0.99018773, 0.99955841, 0.99806449,
       0.99801048, 0.99766623, 0.99891328, 0.98361667, 0.99997919])

In [30]:
logreg_stats_pred_class = [('No' if prob < 0.5 else 'Yes') for prob in logreg_stats_pred_prob ]
logreg_stats_pred_class[:10]

['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']

In [31]:
# import estimator metrics 
from sklearn import metrics

# confusion matrix 
conf_mat = metrics.confusion_matrix(Default.default.astype(str), logreg_stats_pred_class)
print(conf_mat)

[[  40 9627]
 [ 105  228]]
